<a href="https://colab.research.google.com/github/juhee3199/DeepLearning-keras/blob/main/ch8_1_text_generation_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8.1 LSTM으로 텍스트 생성하기

- 딥러닝에서 시퀀스 데이터를 생성하는 일반적인 방법:

이전 토큰을 입력으로 사용해서 시퀀스의 다음 1개 또는 몇개의 토큰을(RNN이나 convnet)으로 예측한다.

- 언어모델을 이용한 텍스트 생성과정
  - 언어 모델: 이전 토큰들이 주어졌을 때 다음 토큰의 확률을 모델링할 수 있는 네트워크
  - 언어모델을 훈련한 후, 이 모델에서 샘플링을 할 수 있다. (샘플링: 새로운 시퀀스를 생성.)

  1) 언어모델에 초기 텍스트 문자열을 주입하고 새로운 글자나 단어를 생성한다.
  - 언어모델은 다음글자의 확률분포를 출력하고, 샘플링을 통해 새로운 단어를 생성한다. 

  2) 생성된 출력은 다시 입력 데이터로 추가되고, 위 과정을 여러번 반복한다.

  3) 반복을 통해 모델이 훈련한 데이터 구조가 반영된 임의의 길이를 가진 시퀀스를 생성할 수 있다. 


In [ ]:
# 예시
# 초기 텍스트 문자열 = 'The cat sat on the m'
# 샘플링을 통해 생성된 다음 글자 = 'a'

# 'a'를 입력데이터로 추가하여 문자열 'The cat sat on the ma' 를 입력모델에 주입
# 샘플링을 통해 생성된 다음 글자 = 't'
# 시퀀스 생성. 'The cat sat on the mat' 

#### 8.1.3 샘플링 전략의 중요성

- 확률적 샘플링: 다음 글자의 확률분포에서 샘플링하는 과정에 **무작위성**을 주입하는 방법

작은 엔트로피는 예상가능한 구조를 가진 시퀀스를 생성(실제처럼 보임)

높은 엔트로피는 창의적인 시퀀스를 생성.

다양한 샘플링 전략으로 실험해보아야 한다.

- Softmax temperature 파라미터 사용하기

샘플링에 사용되는 확률분포의 엔트로피를 나타낸다.

샘플링 과정에서 확률의 양을 조절하기 위해사용.


In [1]:
# temperature를 이용해 확률분포의 가중치 변경하기

import numpy as np

# original_distribution: 전체 합이 1인 넘파이 배열
# temperature: 출력 분포의 엔트로피 양을 결정

def reweight_distribution(original_distribution, temperature = 0.5):  
  distribution = np.log(original_distribution) / temperature
  distribution = np.exp(distribution)
  return distribution / np.sum(distribution)

#### LSTM 층을 사용한 예제

텍스트말뭉치(corpus)에서 N개의 글자로 이루어진 문자열을 추출하여 주입.
N+1 번째 글자를 예측하도록 훈련한다.

모델의 출력은 출력 가능한 모든 글자에 해당하는 소프트맥스 값이다. (= 다음 글자의 확률 분포)

In [3]:
# 데이터 전처리

import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
     origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower() # 말뭉치를 소문자로 바꾸기
print('말뭉치 크기:', len(text))

말뭉치 크기: 600893


1. maxlen 길이를 가진 시퀀스를 중복해서 추출
2. 추출된 시퀀스를 원-핫 인코딩으로 변환하고

    크기가(sequences, maxlen, unique_characters) 인 3D 넘파이 배열 x로 합친다.
    
    동시에 훈련 샘플에 상응하는 타깃(다음 시퀀스)를 담은 배열 준비.



In [13]:
# 1. 글자 시퀀스 추출.

maxlen = 60 #60개의 글자로 된 시퀀스를 추출
step = 3 #세글자씩 건너 뛰면서 새로운 시퀀스를 샘플링

sentences = [] #추출한 시퀀스를 담을 리스트
next_chars = [] #타깃을 담을 리스트(=시퀀스 다음 글자)


for i in range(0, len(text)-maxlen, step): #마지막 for문을 돌 때, 즉 i = len(text)-60일때, 마지막까지 남아있는 60개의 글자로된 시퀀스를 추출함.
  sentences.append(text[i: i+maxlen])
  next_chars.append(text[i+maxlen])

print('시퀀스 개수: ', len(sentences))

chars = sorted(list(set(text))) #말뭉치에서 고유한 글자를 담은 리스트
print('고유한 글자: ', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars) #chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리


시퀀스 개수:  200278
고유한 글자:  57


In [31]:
# 2. 시퀀스 벡터화.

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) #(sequences, maxlen, unique_characters) 크기의 3D 넘파이 배열.
y = np.zeros((len(sentences), len(chars)), dtype=np.bool) #훈련 샘플에 상응하는 타깃(다음 시퀀스)을 담는 배열.

print('x.shape: ', x.shape)

# 글자를 원핫 인코딩하여 0과 1의 이진배열로 바꾸기

for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1   #i번째 sentences차원의 t행 char열을 1로.

  y[i, char_indices[next_chars[i]]] = 1


x.shape:  (200278, 60, 57)


In [28]:
x[:1]

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])

In [22]:
len(x[x==1])

12016680

- 네트워크 구성

하나의 LSTM층 + Dense 분류기

분류기는 가능한 모든 글자에 대한 소프트맥스 출력을 만든다. 


In [40]:
# 다음 글자를 예측하기 위한 단일 LSTM모델
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax')) # 가능한 모든 글자의 소프트맥스

# 모델 컴파일 설정
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [41]:
# 모델의 예측이 주어졌을 때, 새로운 글자를 샘플링

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1) # np.random.multinomial() 다항분포를 시뮬레이션

    return np.argmax(probas)  


In [42]:
# 훈련하여 텍스트 생성

import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):
  model.fit(x, y, batch_size=128, epochs =1) #데이터에서 한번만 반복해서 모델 학습

  seed_text = text[start_index: start_index + maxlen] # 무작위로 시드 텍스트를 선택
  print('--시드 텍스트: "' + seed_text + '"')

  for temperature in [0.2, 0.5, 1.0, 1.2]: # 여러가지 샘플링 temperature를 시도
    print('----temperature: ', temperature)
    generated_text = seed_text
    sys.stdout.write(generated_text) #개행없이 print. 모든 문자열이 연달아 출력.


    # 시드 텍스트에서 시작해서 400개의 글자를 생성
    for i in range(400): 
      # 지금까지 생성된 글자를 원-핫 인코딩으로 바꾸기.
      sampled = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] =1.

      # 다음 글자를 샘플링
      preds = model.predict(sampled, verbose=0)[0]
      next_index = sample(preds, temperature) #sample함수를 통해 다음 글자를 샘플링.
      next_char = chars[next_index]

      generated_text += next_char           
      generated_text = generated_text[1:]


      sys.stdout.write(next_char)
      sys.stdout.flush()

    print()

1565/1565 [==============================] - 262s 167ms/step - loss: 2.2503
--시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
----temperature:  0.2
the slowly ascending ranks and classes, in which,
through for the greet will the complear and invent of the self-its have have have the have and dengerous of the gree dere and the seence of the stand it is and the propossion and the are in the are and the still the seen have the will the same the soulting it is all and soul, and the conderes of the paristance of the way the have the have the seeptes of the good and sould to and the great of the self as a p
----temperature:  0.5
the slowly ascending ranks and classes, in which,
through for only orwer have have the belowicy of the remay and the god in
its still we dengers" of like of the formest man the and and something refere of the enterenties and the most have heartest in the person the against in something accompate of way as the are of the enever of the there or t

KeyboardInterrupt: ignored